In [20]:
import pandas as pd
import numpy as np
import mysql.connector

In [21]:
# Creamos la conexión con la base de datos
conn = mysql.connector.connect(user='root', password='', host='localhost', database='bgg')

In [22]:
# Obtenemos un dataframe con todos los datos de la tabla de juegos

sql_query = "SELECT * FROM ratings_10_with_types"
df = pd.read_sql_query(sql_query, conn)

In [23]:
df.head()

,username,gameId,gameName,rating,date,status,comment
0,-Morphling-,173346,7 Wonders Duel 1,8.0,Nov 2020,Owned,b''
1,-Morphling-,124742,Android: Netrunner 1,9.0,Nov 2020,,"b""I have spent way more time watching other pe..."
2,-Morphling-,205637,Arkham Horror: The Card Game 2,9.0,Nov 2020,,"b""It's a testament to the rest of the game tha..."
3,-Morphling-,230802,Azul 1,7.0,Nov 2020,,"b""It's a solid game with good components, but ..."
4,-Morphling-,760,Battle Line 2,7.0,Jun 2021,Owned,b''


In [24]:
df.columns

Index(['username', 'gameId', 'gameName', 'rating', 'date', 'status',
       'comment'],
      dtype='object')

# BoardGameGeek - Ratings

El fichero `bgg_ratings.csv` contiene los valoraciones que han hecho los usuarios a los juegos del dataset de [BoardGameGeek](https://boardgamegeek.com/). Los atributos incluidos en este fichero son los siguientes:

- `user_id`: Identifficador del usuario.
- `item_id`: Identificador del juego.
- `rating`: Valoración que ha hecho el usuario entre 1 y 10.
- `timestamp`: Fecha en la que hizo la valoración.
- `status`: Inclusión del estado del juego (por ejemplo si es el dueño del juego).


In [25]:
# Definimos las columnas que queremos que se guarden en el csv
bgg_ratings_cols = ['username', 'gameId',  'rating', 'date', 'status']

In [26]:
bgg_ratings_df = df[bgg_ratings_cols]

In [27]:
bgg_ratings_df

,username,gameId,rating,date,status
0,-Morphling-,173346,8.0,Nov 2020,Owned
1,-Morphling-,124742,9.0,Nov 2020,
2,-Morphling-,205637,9.0,Nov 2020,
3,-Morphling-,230802,7.0,Nov 2020,
4,-Morphling-,760,7.0,Jun 2021,Owned
...,...,...,...,...,...
5241405,shantke,22093,10.0,Nov 2013,Owned
5241406,shantke,22093,10.0,Nov 2013,Owned
5241407,shantke,22093,10.0,Nov 2013,Owned
5241408,shantke,22093,10.0,Nov 2013,Owned


In [39]:
usernames = bgg_ratings_df['username'].unique()
users_df = pd.DataFrame(usernames, columns=['username'])
users_df['userId'] = range(1, len(users_df)+1)

In [55]:
bgg_ratings_df['userId'] = bgg_ratings_df['username'].map(users_df.set_index('username')['userId'])

C:\Users\JJD732~1.506\AppData\Local\Temp/ipykernel_16064/557544774.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bgg_ratings_df['userId'] = bgg_ratings_df['username'].map(users_df.set_index('username')['userId'])


In [56]:
bgg_ratings_df.head()

,username,gameId,rating,date,status,userId
0,-Morphling-,173346,8.0,Nov 2020,Owned,1
1,-Morphling-,124742,9.0,Nov 2020,,1
2,-Morphling-,205637,9.0,Nov 2020,,1
3,-Morphling-,230802,7.0,Nov 2020,,1
4,-Morphling-,760,7.0,Jun 2021,Owned,1


In [57]:
bgg_ratings_df['timestamp'] = pd.to_datetime(bgg_ratings_df['date'], format='%b %Y')

In [59]:
bgg_ratings_cols = ['userId', 'gameId',  'rating', 'timestamp', 'status']

In [60]:

bgg_ratings_df = bgg_ratings_df[bgg_ratings_cols]

In [64]:
bgg_ratings_df.to_csv('../data/bgg_ratings.csv.gzip', index=False, compression='gzip')

In [65]:
# Ratings comments
bgg_comments_df = df[['username', 'gameId', 'comment']]
bgg_comments_df.head()

,username,gameId,comment
0,-Morphling-,173346,b''
1,-Morphling-,124742,"b""I have spent way more time watching other pe..."
2,-Morphling-,205637,"b""It's a testament to the rest of the game tha..."
3,-Morphling-,230802,"b""It's a solid game with good components, but ..."
4,-Morphling-,760,b''


In [66]:
bgg_comments_df['userId'] = bgg_comments_df['username'].map(users_df.set_index('username')['userId'])
bgg_comments_df.head()

C:\Users\JJD732~1.506\AppData\Local\Temp/ipykernel_16064/3074935969.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bgg_comments_df['userId'] = bgg_comments_df['username'].map(users_df.set_index('username')['userId'])


,username,gameId,comment,userId
0,-Morphling-,173346,b'',1
1,-Morphling-,124742,"b""I have spent way more time watching other pe...",1
2,-Morphling-,205637,"b""It's a testament to the rest of the game tha...",1
3,-Morphling-,230802,"b""It's a solid game with good components, but ...",1
4,-Morphling-,760,b'',1


In [67]:
bgg_comments_df[['userId', 'gameId', 'comment']].to_csv('../data/bgg_ratings_comments.csv.gzip', index=False, compression='gzip')